### **Good Programming Practices with Usage of Locks**

#### **Case-1:**
It is highly recommended to write the code for releasing locks inside the `finally` block. The advantage is that the lock will always be released, whether an exception is raised or not, and whether it is handled or not.

```python
l = threading.Lock()
l.acquire()
try:
    # Perform required safe operations
finally:
    l.release()


In [1]:
from threading import *
import time

l = Lock()

def wish(name):
    l.acquire()
    try:
        for i in range(10):
            print("Good Evening:", end='')
            time.sleep(0.2)
            print(name)
    finally:
        l.release()

t1 = Thread(target=wish, args=("Dhoni",))
t2 = Thread(target=wish, args=("Yuvraj",))
t3 = Thread(target=wish, args=("Kohli",))

t1.start()
t2.start()
t3.start()


Good Evening:

Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli



#### **Case-2:**
It is highly recommended to acquire a lock by using the `with` statement. The main advantage of the `with` statement is that the lock will be released automatically once control reaches the end of the `with` block, so we are not required to release it explicitly.

This is similar to using the `with` statement for file handling.

**Example for File:**
```python
with open('demo.txt', 'w') as f:
    f.write("Hello...")
```

**Example for Lock:**
```python
lock = threading.Lock()
with lock:
    # Perform required safe operations
    # Lock will be released automatically
```


In [3]:
from threading import *
import time

lock = Lock()

def wish(name):
    with lock:
        for i in range(10):
            print("Good Evening:", end='')
            time.sleep(2)
            print(name)

t1 = Thread(target=wish, args=("Dhoni",))
t2 = Thread(target=wish, args=("Yuvraj",))
t3 = Thread(target=wish, args=("Kohli",))

t1.start()
t2.start()
t3.start()

Good Evening:

Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Dhoni
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Yuvraj
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli
Good Evening:Kohli



### **Q. What is the advantage of using the `with` statement to acquire a lock in threading?**
The lock will be released automatically once control reaches the end of the `with` block, and we are not required to release it explicitly.


**Note:**  
We can use the `with` statement in multithreading for the following cases:  
1. Lock  
2. RLock  
3. Semaphore  
4. Condition  


### **Case-1: Releasing Locks in `finally` Block**

**Scenario: Banking System - Account Transfer**

When transferring money between two bank accounts, you need to lock the accounts to prevent race conditions. Using a `finally` block ensures that the locks are always released, even if an exception occurs during the transfer.

In [ ]:
from threading import Lock

class BankAccount:
    def __init__(self, balance):
        self.balance = balance
        self.lock = Lock()

    def transfer(self, amount, target_account):
        self.lock.acquire()
        try:
            if self.balance >= amount:
                self.balance -= amount
                target_account.deposit(amount)
                print(f"Transferred {amount}. New balance: {self.balance}")
            else:
                print("Insufficient funds.")
        finally:
            self.lock.release()

    def deposit(self, amount):
        self.lock.acquire()
        try:
            self.balance += amount
            print(f"Deposited {amount}. New balance: {self.balance}")
        finally:
            self.lock.release()

# Example usage
account_a = BankAccount(1000)
account_b = BankAccount(500)

account_a.transfer(300, account_b)

### **Case-2: Using the `with` Statement for Locks**

**Scenario: Logging System**

When writing logs to a file from multiple threads, you need to ensure that only one thread writes at a time to avoid garbled logs. Using the `with` statement makes the lock management cleaner and ensures automatic release.

In [4]:
from threading import Lock, Thread
import time

class Logger:
    def __init__(self, log_file):
        self.log_file = log_file
        self.lock = Lock()

    def log(self, message):
        with self.lock:
            with open(self.log_file, 'a') as f:
                f.write(message + '\n')
            print(f"Logged: {message}")

# Example usage
logger = Logger("application.log")

def write_logs(thread_id):
    for i in range(5):
        logger.log(f"Thread-{thread_id}: Log-{i}")
        time.sleep(1)

t1 = Thread(target=write_logs, args=(1,))
t2 = Thread(target=write_logs, args=(2,))
t3 = Thread(target=write_logs, args=(3,))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

Logged: Thread-1: Log-0
Logged: Thread-2: Log-0
Logged: Thread-3: Log-0
Logged: Thread-3: Log-1
Logged: Thread-2: Log-1
Logged: Thread-1: Log-1
Logged: Thread-1: Log-2
Logged: Thread-2: Log-2
Logged: Thread-3: Log-2
Logged: Thread-1: Log-3
Logged: Thread-3: Log-3
Logged: Thread-2: Log-3
Logged: Thread-2: Log-4
Logged: Thread-3: Log-4
Logged: Thread-1: Log-4


1. **Case-1 (finally block)**:
   - Suitable for scenarios where lock handling is complex, or multiple operations are grouped together.
   - Explicit control over when the lock is released, ensuring it's always released even in error situations.

2. **Case-2 (with statement)**:
   - Simplifies lock management and is cleaner for simpler use cases.
   - Automatically releases the lock at the end of the block without requiring explicit `finally`.